In [ ]:
from pathlib import Path

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "torch"
import keras
from PIL import Image

In [ ]:
root_folder = Path("original_images")

In [ ]:
root_folder.exists()

In [ ]:
model = keras.models.load_model("models/safety_ai.keras", compile=False)

In [ ]:
import numpy as np
with_helmet_folder = Path("with_helmet")
without_helmet_folder = Path("without_helmet")
uncertain_folder = Path("uncertain")
img_files = [file for file in root_folder.iterdir()]

In [ ]:
# img_files = np.random.choice(img_files, size=500, replace=False)

In [ ]:
from PIL import Image, ImageOps

def img_process(img_path: Path) ->Image.Image:
    img = Image.open(img_path)
    if img.width > img.height:
        img = img.transpose(Image.Transpose.ROTATE_90)
    width, height = img.size
    pad_size = (height - width) // 2
    padding = (pad_size, 0, height - width- pad_size, 0)
    img = ImageOps.expand(img, padding, fill=0)
    img = img.resize((512, 512))
    return img

In [ ]:
imgs_np = np.array([np.array(img_process(file)) for file in img_files])

In [ ]:
imgs_np.shape

In [ ]:
result = model.predict(imgs_np)

In [ ]:
folders = [uncertain_folder, with_helmet_folder, without_helmet_folder]
for index, r in enumerate(result.argmax(axis=1)):
    folder = folders[r]
    img_file = img_files[index]
    tgt_file = folder / img_file.name
    tgt_file.parent.mkdir(exist_ok=True, parents=True)
    img_file.rename(tgt_file)